In [1]:
import os
import PyPDF2 as pypdf
#import pdfminer
import pandas as pd
import re

#data_dir = '../data'
data_dir = '../Albuquerque2/'

In [2]:
def get_court(text):
    pypdf2_text.split("\n")[1]
    return court

def get_acts(text):
    """
    takes the text of a court case pdf and returns a dictionary of the
    acts (prostitution, pimping, pandering, commercial sex act, sex trafficking) and their
    occurence numbers in the court document
    """
    return acts_dict

# takes the text of a court case pdf and returns wh
def get_elements(text):
    """
    takes the text of a court case pdf and returns a dictionary of the
    elements (force, fraud, coercion, commercial sex act) and their
    occurence numbers in the court document organized by ocurrence descending
    """
    return df

# takes the text of a court case pdf and returns which court heard the case
def get_arresting_dept(text):
    """
    takes the text of a court case pdf and returns a dictionary of the
    possible arresting agencies (agency:ocurrence), to be used in determining arresting agency
    for affidavit requests
    """
    return df

def get_courtcode(text, code):
    return text.count(code)

def get_alb_pd(text):
    return text.count("Albuquerque Police") + text.count("Albuquerque PD") 

def get_all_pd(text):
    return text.count("Police Department") + text.count("PD") + text.count("Police Dep") 

def get_defendant_name(text):
    
    return name

"""
def get_1591(text): 
    if "1591" in text:
        occurences = text.count("1591")
    else:
        occurences = 0
    return occurences

def get_2421(text): 
    if "2421" in text:
        occurences = text.count("2421")
    else:
        occurences = 0
    return occurences

def get_1594(text): 
    if "1594" in text:
        occurences = text.count("1594")
    else:
        occurences = 0
    return occurences

def get_2422(text): 
    if "2422" in text:
        occurences = text.count("2422")
    else:
        occurences = 0
    return occurences

def get_2423(text): 
    if "2423" in text:
        occurences = text.count("2423")
    else:
        occurences = 0
    return occurences

def get_1328(text): 
    if "1328" in text:
        occurences = text.count("1328")
    else:
        occurences = 0
    return occurences

def get_1952(text): 
    if "1952" in text:
        occurences = text.count("1952")
    else:
        occurences = 0
    return occurences

def get_word(text, word):
    if word.str() in text: 
        occurences = text.count(word.str())
    else:
        occurences = 0 
    return occurences 
    
"""

print("")

In [4]:
files = []
c1591 = []
c2421 = []
c1594 = []
c2422 = []
c2423 = []
c1328 = []
c1952 = []
alb_pd = []
all_pd = []

for filename in os.scandir(data_dir):
        if ".pdf" in filename.name:
            reader = pypdf.PdfReader(filename.path)
            pypdf2_text = ""
            for i in range(len(reader.pages)):
                pypdf2_text += reader.pages[i].extract_text()  
            files.append(filename.name)
            #c1591.append(get_courtcode(pypdf2_text, "1591"))
            #c2421.append(get_courtcode(pypdf2_text, "2421"))
            #c1594.append(get_courtcode(pypdf2_text, "1594"))
            #c2422.append(get_courtcode(pypdf2_text, "2422"))
            #c2423.append(get_courtcode(pypdf2_text, "2423"))
            #c1328.append(get_courtcode(pypdf2_text, "1328"))
            #c1952.append(get_courtcode(pypdf2_text, "1952"))
            alb_pd.append(get_alb_pd(pypdf2_text))
            all_pd.append(get_all_pd(pypdf2_text))
        else:
            continue

In [5]:
len(files), len(c1952)

(407, 0)

In [8]:
cases = pd.DataFrame({'file_name': files,
                      #'Code_1591': c1591,
                      #'Code_2421': c2421, 
                      #'Code_1594': c1594, 
                      #'Code_2422': c2422, 
                      #'Code_2423': c2423, 
                      #'Code_1328': c1328, 
                      #'Code_1952': c1952, 
                     'AlbPD_Mentions':alb_pd, 
                     'AllPD_Mentions': all_pd})
len(cases[cases['AlbPD_Mentions']>0])

50

In [9]:
cases[cases['AlbPD_Mentions']>0]

,file_name,AlbPD_Mentions,AllPD_Mentions
23,"Brewer v. City of Albuquerque, 18 F.4th 1205.pdf",3,7
68,Grossete v. City of Albuquerque Open Space Pol...,1,18
69,"Hall v. State of N.M. Child., Youth and Fams. ...",1,10
81,"J.H. v. Bernalillo County, 61 F. Supp. 3d 1085...",1,2
86,"Johnson v. State, 2019 Ark. 391.pdf",2,8
112,"Parsons v. Velasquez, 551 F. Supp. 3d 1085.pdf",17,66
118,"Plummer v. Town of Dickson, 2013 U.S. Dist. LE...",1,14
136,"Semsroth v. City of Wichita, 2007 U.S. Dist. L...",2,89
159,"State v. Lassiter, 2016-NMCA-078.pdf",1,1
160,"State v. Olson, 2012-NMSC-035.pdf",1,2


In [25]:
code_cols = cases.columns[1:8]

In [26]:
cases['TotalCharges'] = cases[code_cols].sum(axis = 1)

In [27]:
cases_sub = cases[(cases.TotalCharges > 0) & (cases.AllPD_Mentions > 0) ]

In [28]:
len(cases_sub)

42

In [29]:
cases_sub.to_csv('../Albuquerque/FilteredAlbuquerque.csv')

# Seth working on names

In [ ]:
'''
files = []
c1591 = []
c2421 = []
c1594 = []
c2422 = []
c2423 = []
c1328 = []
c1952 = []
alb_pd = []
all_pd = []
'''
for filename in os.scandir(data_dir):
        if ".pdf" in filename.name:
            reader = pypdf.PdfReader(filename.path)
            pypdf2_text = ""
            for i in range(len(reader.pages)):
                pypdf2_text += reader.pages[i].extract_text()  
            files.append(filename.name)
            
            '''
            c1591.append(get_courtcode(pypdf2_text, "1591"))
            c2421.append(get_courtcode(pypdf2_text, "2421"))
            c1594.append(get_courtcode(pypdf2_text, "1594"))
            c2422.append(get_courtcode(pypdf2_text, "2422"))
            c2423.append(get_courtcode(pypdf2_text, "2423"))
            c1328.append(get_courtcode(pypdf2_text, "1328"))
            c1952.append(get_courtcode(pypdf2_text, "1952"))
            alb_pd.append(get_alb_pd(pypdf2_text))
            all_pd.append(get_all_pd(pypdf2_text))
            '''
        else:
            continue

In [ ]:
cases = pd.DataFrame({'file_name': files,
                      'Code_1591': c1591,
                      'Code_2421': c2421, 
                      'Code_1594': c1594, 
                      'Code_2422': c2422, 
                      'Code_2423': c2423, 
                      'Code_1328': c1328, 
                      'Code_1952': c1952, 
                     'AlbPD_Mentions':alb_pd, 
                     'AllPD_Mentions': all_pd})
cases

# break
-------------------------------------------------------------

In [30]:
cases

,file_name,Code_1591,Code_2421,Code_1594,Code_2422,Code_2423,Code_1328,Code_1952,AlbPD_Mentions,AllPD_Mentions,TotalCharges
0,"A.M. v. N.M. Dep_t of Health, 148 F. Supp. 3d ...",0,0,0,0,0,0,0,0,0,0
1,"ACLU of N.M. v. City of Albuquerque, 139 N.M. ...",0,0,0,0,0,0,0,1,12,0
2,"Aguilar v. McAleenan, 2019 U.S. Dist. LEXIS 19...",0,0,0,0,0,0,0,0,0,0
3,"Allen v. City of Albuquerque, 2015 U.S. Dist. ...",0,0,0,0,0,0,0,0,3,0
4,"Alvarado v. KOB-TV, L.L.C., 493 F.3d 1210.pdf",0,0,0,0,0,0,0,3,6,0
...,...,...,...,...,...,...,...,...,...,...,...
266,"Webb v. Padilla, 2009 U.S. Dist. LEXIS 101489.pdf",0,0,0,0,0,0,0,3,8,0
267,"White v. Town of Hurley, 2019 U.S. Dist. LEXIS...",0,0,0,0,0,0,0,0,39,0
268,"Womack v. Unified Gov_t, 2021 U.S. Dist. LEXIS...",0,0,0,0,0,0,0,0,17,0
269,"Young v. Wilham, 2017-NMCA-087.pdf",0,0,0,0,0,1,0,1,42,1


In [31]:
# Read in the text from case PDFs and store in dataframe
cases = pd.DataFrame({'file_name': pd.Series(dtype='str'),
                      #'court': pd.Series(dtype='str'),                      
                      #'acts': pd.Series(dtype='str'),
                      #'elements_used': pd.Series(dtype='str'),
                      #'location_likely': pd.Series(dtype='str'),
                      #'arresting_department_likely': pd.Series(dtype='float'), 
                      'Code_1591': pd.Series(dtype='int'), 
                      'Code_2421': pd.Series(dtype='int'), 
                      'Code_1594': pd.Series(dtype='int'), 
                      'Code_2422': pd.Series(dtype='int'), 
                      'Code_2423': pd.Series(dtype='int'), 
                      'Code_1328': pd.Series(dtype='int'), 
                      'Code_1952': pd.Series(dtype='int')})

In [32]:
for filename in os.scandir(data_dir):
        if ".pdf" in filename.name:
            df = pd.DataFrame({'file_name': pd.Series(dtype='str'),
                                #'court': pd.Series(dtype='str'),                      
                                #'acts': pd.Series(dtype='str'),
                                #'elements_used': pd.Series(dtype='str'),
                                #'location_likely': pd.Series(dtype='str'),
                                #'arresting_department_likely': pd.Series(dtype='float'), 
                                  'Code_1591': pd.Series(dtype='int'), 
                                  'Code_2421': pd.Series(dtype='int'), 
                                  'Code_1594': pd.Series(dtype='int'), 
                                  'Code_2422': pd.Series(dtype='int'), 
                                  'Code_2423': pd.Series(dtype='int'), 
                                  'Code_1328': pd.Series(dtype='int'), 
                                  'Code_1952': pd.Series(dtype='int')})
            '''
            if "Attachment" in filename:
                main_document_name = re.sub("_Attachment[0-9]+", "", filename.name)
                if (cases['file_name'].eq(main_document_name)).any():
                    # TODO (can we scrape the attachments because they are often screenshots)
                    # add the attachment and what it is to the dictionary
                    attachment_type = get_attachment_type()
                else:
                    # TODO
                    # create a new pandas column for the main court file we extracted and add this attachment as the first
                    # in the attachment dict, then 
            else:
            '''
            reader = pypdf.PdfReader(filename.path)
            pypdf2_text = ""
            for i in range(len(reader.pages)):
                pypdf2_text += reader.pages[i].extract_text() 
            df['file_name'] = [filename.name]
            #df['court'] = get_court(pypdf2_text)
            #df['acts'] = get_acts(pypdf2_text)
            #df['elements_used'] = get_elements(pypdf2_text)
            #df['arresting_department_likely'] = get_arresting_dept(pypdf2_text)
            df['Code_1591'] = [get_courtcode(pypdf2_text, "1591")]
            df['Code_2421'] = [get_courtcode(pypdf2_text, "2421")]
            df['Code_1594'] = [get_courtcode(pypdf2_text, "1594")]
            df['Code_2422'] = [get_courtcode(pypdf2_text, "2422")]
            df['Code_2423'] = [get_courtcode(pypdf2_text, "2423")]
            df['Code_1328'] = [get_courtcode(pypdf2_text, "1328")]
            df['Code_1952'] = [get_courtcode(pypdf2_text, "1952")]
            cases = cases.append(df)


C:\Users\Seth\AppData\Local\Temp\ipykernel_328\73401225.py:45: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cases = cases.append(df)
C:\Users\Seth\AppData\Local\Temp\ipykernel_328\73401225.py:45: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cases = cases.append(df)
C:\Users\Seth\AppData\Local\Temp\ipykernel_328\73401225.py:45: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cases = cases.append(df)
C:\Users\Seth\AppData\Local\Temp\ipykernel_328\73401225.py:45: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cases = cases.append(df)
C:\Users\Seth\AppData\Local\Temp\ipykernel_328\73401225.py:45: FutureWarning: The frame.append method is dep

KeyboardInterrupt: 

In [ ]:
cases

In [35]:
# messing with pypdf
df = pd.DataFrame({'file_name': pd.Series(dtype='str'),
                    'acts': pd.Series(dtype='str'),
                    'elements_used': pd.Series(dtype='str'),
                    'location_likely': pd.Series(dtype='str'),
                    'arresting_department_likely': pd.Series(dtype='float')})
df['file_name'] = "A.M. v. N.M. Dep't of Health, 148 F. Supp. 3d 1232.pdf"
reader = pypdf.PdfReader("../data/A.M. v. N.M. Dep't of Health, 148 F. Supp. 3d 1232.pdf")
pypdf2_text = ""
for i in range(len(reader.pages)):
    pypdf2_text += reader.pages[i].extract_text() 

FileNotFoundError: [Errno 2] No such file or directory: "./data/A.M. v. N.M. Dep't of Health, 148 F. Supp. 3d 1232.pdf"

In [36]:
print(pypdf2_text)

 Bledsoe v. Carreno
United States Court of Appeals for the Tenth Circuit
November 15, 2022, Filed
No. 20-3252
Reporter
53 F.4th 589 *; 2022 U.S. App. LEXIS 31579 **; 2022 WL 16942631
FLOYD S. BLEDSOE, Plaintiff - Appellee, v. RANDY CARRENO; TROY FROST; JEFFREY HERRIG; ROBERT 
POPPA, Defendants - Appellants, BOARD OF COUNTY COMMISSIONERS FOR THE COUNTY OF JEFFERSON, 
KANSAS; GEORGE JOHNSON; JIM WOODS; TERRY MORGAN; MICHAEL HAYES; UNKNOWN OFFICERS OF 
THE JEFFERSON COUNTY SHERIFF'S DEPARTMENT; UNKNOWN OFFICERS OF THE KANSAS BUREAU OF 
INVESTIGATION; JIM VANDERBILT, Defendants.
Prior History:  [**1] Appeal from the United States District Court for the District of Kansas. (D.C. No. 2:16-CV-
02296-DDC-GLR).
Bledsoe v. Bd. of Cnty. Comm'rs, 501 F. Supp. 3d 1059, 2020 U.S. Dist. LEXIS 215485, 2020 WL 6781389 (D. 
Kan., Nov. 18, 2020)
Core Terms
murder, fabricating, qualified immunity, conspiracy, allegations, failure-to-intervene, constitutional violation, district 
court, deprivation, confes

AttributeError: 'NoneType' object has no attribute 'group'

In [40]:
pypdf2_text

' Bledsoe v. Carreno\nUnited States Court of Appeals for the Tenth Circuit\nNovember 15, 2022, Filed\nNo. 20-3252\nReporter\n53 F.4th 589 *; 2022 U.S. App. LEXIS 31579 **; 2022 WL 16942631\nFLOYD S. BLEDSOE, Plaintiff - Appellee, v. RANDY CARRENO; TROY FROST; JEFFREY HERRIG; ROBERT \nPOPPA, Defendants - Appellants, BOARD OF COUNTY COMMISSIONERS FOR THE COUNTY OF JEFFERSON, \nKANSAS; GEORGE JOHNSON; JIM WOODS; TERRY MORGAN; MICHAEL HAYES; UNKNOWN OFFICERS OF \nTHE JEFFERSON COUNTY SHERIFF\'S DEPARTMENT; UNKNOWN OFFICERS OF THE KANSAS BUREAU OF \nINVESTIGATION; JIM VANDERBILT, Defendants.\nPrior History:  [**1] Appeal from the United States District Court for the District of Kansas. (D.C. No. 2:16-CV-\n02296-DDC-GLR).\nBledsoe v. Bd. of Cnty. Comm\'rs, 501 F. Supp. 3d 1059, 2020 U.S. Dist. LEXIS 215485, 2020 WL 6781389 (D. \nKan., Nov. 18, 2020)\nCore Terms\nmurder, fabricating, qualified immunity, conspiracy, allegations, failure-to-intervene, constitutional violation, district \ncourt,